# REST API Consumer Demo
This demo will consume our trained model through a hosted REST API.

In [ ]:
import warnings

import numpy as np
import pandas as pd
import requests
from pandas import DataFrame

from ae5_tools import demand_env_var

warnings.filterwarnings("ignore")
np.random.seed(42)

In [ ]:
"""
Load user specific configuration.
"""

from ae5_tools import load_ae5_user_secrets

load_ae5_user_secrets()

In [ ]:
"""
Invokes the REST Endpoint
"""


def invoke_rest_endpoint(input_data: dict) -> dict:
    endpoint_url: str = demand_env_var(name="SELF_HOSTED_MODEL_ENDPOINT")
    response = requests.post(url=f"{endpoint_url}/invocations", json=input_data, verify=False)
    if response.status_code != 200:
        raise Exception(f"Received status code: ({response.status_code}), Failed to call prediction: {response.text}")
    return response.json()

In [ ]:
"""
Data Preparation
Loads the data from csv file, and returns our train, test splits for training.
"""


def prepare_data(csv_url: str) -> dict:
    csv_data: pd.DataFrame = pd.read_csv(csv_url, sep=",")

    # The predicted column is "quality" which is a scalar from [3, 9]
    data_x = csv_data.drop(["quality"], axis=1)
    data_y: DataFrame = csv_data[["quality"]]

    return {"data_x": data_x, "data_y": data_y}

In [ ]:
"""
Get prediction for the  given input.
"""


def predict(data_x: pd.DataFrame) -> pd.DataFrame:
    # Build the prediction request
    payload: list[dict] = data_x.to_dict(orient="records")
    json_payload: dict = {"dataframe_records": payload}

    # Call prediction service
    predictions: dict = invoke_rest_endpoint(input_data=json_payload)
    return pd.DataFrame(predictions)

# Get predictions for data set

In [ ]:
"""
Load a data set to predict wine quality on.
"""

csv_url: str = "datasets/winequality-red.csv"
data: dict = prepare_data(csv_url=csv_url)
data_x: pd.DataFrame = data["data_x"]
data_y: pd.DataFrame = data["data_y"]

predictions_df: pd.DataFrame = predict(data_x=data_x)
predictions_df["truth"] = data_y

In [ ]:
predictions_df